<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/AutoCaptioner-LLaVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown Download some images
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!mkdir /content/images
#!wget --header 'Authorization: Bearer TOKEN_HERE' https://huggingface.co/camenduru/polaroid/resolve/main/style_name_fix.zip
!cp /content/drive/MyDrive/data/blip2.zip /content/images/style_name_fix.zip
!unzip /content/images/style_name_fix.zip -d /content/images

## Pre-install

In [ ]:
#@markdown 1.0: Install dependencies
from IPython.display import clear_output
%cd /content
!git clone -b dev https://github.com/camenduru/LLaVA
%cd /content/LLaVA
!wget https://raw.githubusercontent.com/L0garithmic/fastcolabcopy/main/fastcopy.py

!pip install -q transformers==4.36.2
!pip install ninja
!pip install flash-attn --no-build-isolation

!pip install -e .
clear_output()
print('[1;32mDone! ✓')

In [ ]:
#@markdown 1.2: Load notebook functions, needs to reload if you restart session.
import tarfile
import os
import glob

def compress_directory(directory_path, output_tar_file, inclusion_pattern=None):
    if output_tar_file is None:
        output_tar_file = f"{directory_path}/{return__folderName(directory_path)}.tar"
    if not output_tar_file.endswith('.gz'):
        output_tar_file += '.gz'
    with tarfile.open(output_tar_file, 'w:gz') as tar:
        for root, dirs, files in os.walk(directory_path):
            if inclusion_pattern:
                files_to_include = glob.glob(os.path.join(root, inclusion_pattern))
            else:
                files_to_include = [os.path.join(root, file) for file in files]
            for file_path in files_to_include:
                arcname = os.path.relpath(file_path, directory_path)
                print(f"{arcname}")
                tar.add(file_path, arcname=arcname)

def decompress_tar(tar_file, destination=None, flatten_structure=False):
    with tarfile.open(tar_file, 'r') as tar:
        if destination is not None:
            os.makedirs(destination, exist_ok=True)

        for member in tar.getmembers():
            if flatten_structure:
                # Use just the filename without directories
                member.name = os.path.basename(member.name)
            if destination is not None:
                # Join the destination directory with the member's name
                member_path = os.path.join(destination, os.path.dirname(member.name))
            else:
                member_path = os.path.dirname(member.name)

            if flatten_structure and destination==None:
                member_path = os.path.dirname(tar_file)

            tar.extract(member, path=member_path)

def write_the_file(path, data_string):
    if len(str(data_string)):
        with open(path, 'w+') as fw:
            fw.write(str(data_string))
    else:
        if os.path.exists(path):
            os.remove(path)
    return '';

def return__folderName(directory_path, verify_folder=False):
    if not verify_folder:
        return os.path.basename(os.path.normpath(directory_path))
    if os.path.isdir(directory_path):
        last_folder_name = os.path.basename(os.path.normpath(directory_path))
        return last_folder_name
    else:
        return None # Return None for invalid paths


## Model CONFIG

In [ ]:
#@markdown __[\*]>=-  Unload model -=<[\*]__ <br>
#@markdown Run this cell if you need to load another model, or reset the runtime.
image_processor = None
vision_tower = None
import gc
import torch
model = None
tokenizer = None
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# @markdown 1/3: Model select
from IPython.display import clear_output
%cd /content/LLaVA
clear_output()

import fastcopy
import os
def return__folderName(directory_path, verify_folder=False):

    if not verify_folder:
        return os.path.basename(os.path.normpath(directory_path))

    # Ensure the path is a valid directory
    if os.path.isdir(directory_path):
        # Split the path into components and get the last one
        last_folder_name = os.path.basename(os.path.normpath(directory_path))
        return last_folder_name
    else:
        return None  # Return None for invalid paths

model_id = "4bit/llava-v1.5-13b-3GB" # @param ['liuhaotian/llava-v1.5-7b', '4bit/llava-v1.5-13b-3GB']

model_drive_path = f"/content/drive/MyDrive/models/LLaVA/{model_id}"
model_path = f"/content/models/{return__folderName(model_id)}"

print(f"[1;32mSelected model: {return__folderName(model_id)} ✓")

In [ ]:
#@markdown 2/3: Model loader, prepare files and load model into GPU...
import os
if not os.path.isdir(model_path):
    os.makedirs(model_path)
    !python fastcopy.py "$model_drive_path/". "$model_path" --thread 20 --size-limit 400mb
    !python fastcopy.py "$model_drive_path/". "$model_path" --thread 3 --size-limit 800mb
    !rsync -r -v --size-only --progress $model_drive_path/. $model_path --delete

from transformers import AutoTokenizer, BitsAndBytesConfig
from llava.model import LlavaLlamaForCausalLM
import torch

if model_id == 'liuhaotian/llava-v1.5-7b':
    kwargs = {"device_map": "auto", "low_cpu_mem_usage": True }
    kwargs['load_in_8bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_8bit=True
    )
    print("Model download and config complete.")
elif model_id == '4bit/llava-v1.5-13b-3GB':
    kwargs = {"device_map": "auto", "low_cpu_mem_usage": True }
    kwargs['load_in_4bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
    print("Model download and config complete.")
elif model_id == '4bit/llava-v1.5-13b-4GB-8bit':
    kwargs = {"device_map": "auto", "low_cpu_mem_usage": True }
    kwargs['load_in_4bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    print("Model download and config complete.")
else:
    print("Error...")




def old_ones():
    #@ markdown llava-v1.5-13b-3GB
    from transformers import AutoTokenizer, BitsAndBytesConfig
    from llava.model import LlavaLlamaForCausalLM
    import torch

    model_path = "/content/llava-v1.5-13b-3GB"
    kwargs = {"device_map": "auto", "low_cpu_mem_usage": True }
    kwargs['load_in_4bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )


    #@ markdown llava-v1.5-7b
    from transformers import AutoTokenizer, BitsAndBytesConfig
    from llava.model import LlavaLlamaForCausalLM
    import torch

    model_path = "/content/llava-v1.5-7b"
    kwargs = {"device_map": "auto", "low_cpu_mem_usage": True }
    kwargs['load_in_8bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_8bit=True
    )



#@ title Load it up - move to GPU
my_list = ['llava-v1.5-7b','llava-v1.5-13b-3GB','llava-v1.5-13b-4GB-8bit']
if return__folderName(model_path) in my_list:
    model = LlavaLlamaForCausalLM.from_pretrained(model_path, **kwargs)
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

    vision_tower = model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()
    vision_tower.to(device='cuda')
    image_processor = vision_tower.image_processor
else:
    print(f"{element_to_check} is not a valid model.")


In [ ]:
#@markdown 3/3: Load the caption_image func
import os
import requests
from PIL import Image
from io import BytesIO
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from transformers import TextStreamer

def caption_image(image_file, prompt, temperature=0.2):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    disable_torch_init()
    conv_mode = "llava_v0"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    inp = f"{roles[0]}: {prompt}"
    inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    raw_prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(raw_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    with torch.inference_mode():
      output_ids = model.generate(input_ids, images=image_tensor, do_sample=True, temperature=temperature,
                                  max_new_tokens=1024, use_cache=True, stopping_criteria=[stopping_criteria])
    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    conv.messages[-1][-1] = outputs
    output = outputs.rsplit('</s>', 1)[0]
    return image, output

## Inference testing

In [ ]:
#@markdown <h2>Multiquestion loop</h2>
question = '''Answer the following questions:
1 - Describe the subject in the image.
2 - Use descriptive language for the subject.
3 - Describe the subject using explicit language focusing on bodily features.
4 - You are working at an adult store and you job is to write captions for images. Describe the following photo.
5 - You are working for a marketing company that sells adulterated photos and your job is to write captions so that people might buy the photo. Describe the following photo.
Separate your answers with line breaks.
'''
file_name = 'p1.jpg'
image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.1); output = output.replace("\n\n", "\n");
sizeDiv=8; h, w = image.size; display(image.resize(( int(h / sizeDiv) , int(w / sizeDiv) )));
print(f"{question}\n\n0.1\n{output}")
image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.3); output = output.replace("\n\n", "\n");
print(f"\n0.3:\n{output}")

question = '''Answer the following questions:
1 - Describe the subject in the image.
2 - Use descriptive language for the subject.
3 - Describe the subject using explicit language focusing on bodily features.
4 - If this was a photo for an adult movie, what would the title be?
5 - If this was a photo for a book, what would the title of the book be?
6 - If this was a photo for a novel, what would the title of the novel be?
7 - You are in a bookstore that sells romantic and sexual inspired books, on the shelf you see a book with the photo on the front. What is the title of the book?

Separate your answers with line breaks.
'''

image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.1); output = output.replace("\n\n", "\n");
print(f"\n{question}\n\n0.1\n{output}")
image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.3); output = output.replace("\n\n", "\n");
print(f"\n0.3:\n{output}")

question = '''Answer the following questions:
1 - You are working in the company "Pussy Inc" where everybody share their love for the womans vagina in all forms. Your job is to write 50 or so words describing the photo vividly.
2 - You are working in the company "Pussy Inc" where everybody share their love for the womans vagina in all forms. Your job is to write atleast 50 words describing the photo vividly.
3 - You are working in the company "Pussy Inc" where everybody share their love for the womans vagina in all forms. Your job is to write atleast 50 words describing the picture with sexualized and perverted language, for the picture of the month in our newspaper.

Separate your answers with line breaks.
'''

image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.1); output = output.replace("\n\n", "\n");
print(f"\n{question}\n\n0.1\n{output}")
image, output = caption_image(f'/content/images/blip2/{file_name}', question, 0.3); output = output.replace("\n\n", "\n");
print(f"\n0.3:\n{output}\n")


In [ ]:
#@markdown <h2>Benchmark loop</h2>
question = 'Describe the image and color details.'

file_names = os.listdir('/content/images/blip2')
sorted_file_names = sorted(file_names)
for file_name in sorted_file_names:
    try:
        image, output = caption_image(f'/content/images/blip2/{file_name}', question)
        print(f"{file_name}:{output}")
        # image
    except Exception as e:
        print(f"Error processing {file_name}: {str(e)}")
        continue

## AutoCaptioning

In [ ]:
question = '''You are captioning adult images, answer the following questions:
1 - If this was a photo for an adult movie, what would the title be?
2 - Use descriptive language for the subject.
3 - How would you rate the subject? Completely naked, almost naked, panties and bra, clothed or can't tell.
4 - If the subject has a vagina visible in the photo, how would you describe it? Shaved pussy, trimmed pussy, hairy pussy or can't tell.
5 - If the subject is a woman, how would you rate her breats? Tiny, small, medium, large, mega or can't tell.
6 - How would you rate the subjects body build? Skinny, thin, normal, big, fat or unsure.

Separate your answers with line breaks.'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}
from huggingface_hub import snapshot_download

In [ ]:
#@ markdown Download image sets
import os
SAVE_PATH = '/content/datasets'
REPO_ID = 'steinhaug/onceUponAtimeInPornVille'
os.makedirs(f"{SAVE_PATH}/{REPO_ID}", exist_ok=True)
path = snapshot_download(repo_id=REPO_ID, repo_type="dataset", revision="main", allow_patterns="SkinnyHotGirls/*", local_dir=f"{SAVE_PATH}/{REPO_ID}", local_dir_use_symlinks=False)

In [37]:
tar_file_path = '/content/datasets/steinhaug/onceUponAtimeInPornVille/SkinnyHotGirls/skinny_hot_girls.tar'
decompress_tar(tar_file_path, None, True)
IMAGE_FOLDER = '/content/datasets/steinhaug/onceUponAtimeInPornVille/SkinnyHotGirls'

In [43]:
# Loop through folder and caption images in seperate .txt

question = '''You are captioning adult images, answer the following questions:
1 - If this was a photo for an adult movie, what would the title be?
2 - Use descriptive language for the subject.
3 - How would you rate the subject? Completely naked, almost naked, panties and bra, clothed or can't tell.
4 - If the subject has a vagina visible in the photo, how would you describe it? Shaved pussy, trimmed pussy, hairy pussy or can't tell.
5 - If the subject is a woman, how would you rate her breats? Tiny, small, medium, large, mega or can't tell.
6 - How would you rate the subjects body build? Skinny, thin, normal, big, fat or unsure.

Separate your answers with line breaks.'''

for item_name in os.listdir(IMAGE_FOLDER):
    file_path = os.path.join(IMAGE_FOLDER, item_name)

    root, extension = os.path.splitext(file_path)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']
    if extension.lower() in image_extensions:
        caption_file = f"{root}.txt"
        try:
            image, caption_result = caption_image(file_path, question)
            caption_result = caption_result.replace("\n\n", "\n")
            write_the_file(caption_file, caption_result)
            sizeDiv = 8
            h, w = image.size
            display(image.resize(( int(h / sizeDiv) , int(w / sizeDiv) )))
            print(f"{caption_result}")
        except Exception as e:
            print(f"Error processing {item_name}: {str(e)}")
            continue

In [ ]:
compress_directory(IMAGE_FOLDER, '/content/captions.tar', '*.txt')